<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/PySyft_Keystone_2_with_Secure_Training_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(notebook copy from Kaggle)

**Issues with PySyft: (state of 2019, August 20th)** 

1.   Unfortunately, PySyft does not support CUDA. https://github.com/OpenMined/PySyft/issues/1893
That is why I decided for a simple and small dataset. 
2.   https://github.com/OpenMined/PySyft/issues/2425
3.https://github.com/OpenMined/PySyft/issues/2426 
4. "AttributeError: 'Linear' object has no attribute 'fix_prec'"
https://github.com/OpenMined/PySyft/pull/2364
5. Running PySyft code often ends up with an empty AssertionError. It is related to the installation process, because `pip install` could not install succesfully. The current PySyft version does not fit with the current PyTorch, TorchVision versions. 
6. PySyft currenlty only supports exactly 2 workers, neither more no less. 
7. We get error messages, when we train with non-linear models, for example while using Convolutional and BatchNorm layers






Data is shared in an encrypted way across multiple end devices via workers
```(data.fix_precision().share(ada, bob, crypto_provider= crypto_provdr)```

The end device downloads the current model, i.e. the model is shared with the end devices: 
``` model.fix_precision().share(ada,bob, crypto_provider=crypto_provdr)```


The end device improves the model by learning from data on that end device, 
and then summarizes the changes as a small focused update. 

It is just this update to the model that is sent to the cloud (secure worker), using encryption, where it is averaged with other updates (coming from end devices) to improve the shared model. 

All the training data remains on the end devices. The model, the inputs, the model outputs, the weights, etc. will be encrypted as well. 

In [0]:
!pip install syft

In [2]:

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import syft as sy

import os


W0827 19:48:44.512621 140054480209792 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0827 19:48:44.530401 140054480209792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# Visualize plot
def plot_loss_acc(n_epochs, train_losses, valid_losses, valid_accuracies):
    fig, (ax1, ax2) = plt.subplots(figsize=(14,6), ncols=2)
    ax1.plot(valid_losses, label='Validation loss')
    ax1.plot(train_losses, label='Training loss')
    ax1.legend(frameon=False)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    #x_ticks = [x for x in range(0,n_epochs,2)]
    #plt.xticks(x_ticks)
    
    ax2.plot(valid_accuracies, label = 'Validation accuracy')
    ax2.legend(frameon=False)
    ax2.set_xlabel('Epochs')
    
    plt.tight_layout()

In [6]:
# create workers, 
hook = sy.TorchHook(torch)

ada = sy.VirtualWorker(hook, 'ada')
bob = sy.VirtualWorker(hook, 'bob')

crypto_provdr = sy.VirtualWorker(hook, 'crypto_provdr') #gives crypto primitives we may need

W0827 19:48:57.118300 140054480209792 hook.py:102] Torch was already hooked... skipping hooking process


In [7]:
# define the transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5 ), (0.5,0.5,0.5 ))
])

# load the datasets
fulltrainset = datasets.CIFAR10(root='./CIFAR10_data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10('~/.pytorch/CIFAR10_data', download=True, train=False, transform=transform)

train_size = int(len(fulltrainset)* 0.8)
valid_size = len(fulltrainset) - train_size

# split the dataset
trainset, validationset = torch.utils.data.random_split(fulltrainset, [train_size, valid_size])
trainset = trainset.dataset
validationset = validationset.dataset


0it [00:00, ?it/s]

170500096it [00:06, 25740523.05it/s]                               
0it [00:00, ?it/s]

170500096it [00:06, 26959878.12it/s]                               


In [1]:
batch_size = 64
number_train_items = 640
number_valid_items = 256

# we assume that the server has access to some data to first train its model
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# The client has some data and gets predictions on it using the server's model. 
# The client encrypts its data by sharing it additively across the workers ada, bob.
valid_loader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True)


private_train_loader = []
for i, data, target in enumerate(train_loader):
  if i < number_train_items / batch_size:
    private_train_loader.append(
        (data.fix_precision().share(ada, bob, crypto_provider= crypto_provdr),
        target.fix_precision().share(ada, bob, crypto_provider=crypto_provdr)
        ))
    
private_valid_loader = []
for i, data, target in enumerate(valid_loader):
  if i < number_valid_items / batch_size:
    private_valid_loader.append(
        (data.fix_precision().share(ada, bob, crypto_provider= crypto_provdr),
        target.fix_precision().share(ada, bob, crypto_provider=crypto_provdr)
        ))

    
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=True)


len_trainloader = len(private_train_loader)
len_validloader = len(private_valid_loader)
print(len_trainloader)


NameError: ignored

**Architecture**

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x # F.log_softmax(x, dim=1) Do not call log_softmax here, it is not supported yet. 
    


**Functions train, test**

In [0]:
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''
    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        n = m.in_features
        # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0, 1/np.sqrt(n))
        # m.bias.data should be 0
        m.bias.data.fill_(0)
            

def train(args, model, device, train_loader, optimizer, epoch):
    # initialize variables to monitor training and validation loss
    train_loss = 0.0
    correct = 0.0
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader): # <-- now it is a distributed dataset
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        #output = F.log_softmax(output)
        #loss = F.nll_loss(output, target) # NLLLoss does not work with PySyft encryption
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        optimizer.step()
        #print(' \tTrain. Loss: {:.6f}'.format(current_loss))    

        # get the loss per batch and accumulate
        train_loss += loss.item()
        
        
    # calculate the average loss per epoch
    train_loss = train_loss/len(train_loader)
    #print('Epoch: {} \tTrain. Loss: {:.6f}'.format(epoch, train_loss))    
    return train_loss
         

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 32
        self.epochs = 2
        self.lr = 0.001
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


model = Model()
model.apply(weights_init_normal)

optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [3, 10, 14], 0.1)



In [0]:
# perform the encrypted evaluation. The model weights, the data inputs, the prediction and 
# the target used for scoring are encrypted!

def valid_encrypted(args, model, device, loader, optimizer, epoch):
    # initialize variables to monitor training and validation loss
    valid_loss = 0.0
    correct = 0.0
    
    model.eval()
    
    for batch_idx, (data, target) in enumerate(loader): # <-- now it is a distributed dataset
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size

        # get the loss per batch and accumulate
        valid_loss += loss.item()
        
        
    # calculate the average loss per epoch
    valid_loss = valid_loss/len(loader)
    return valid_loss
            

**Start the training**

In [0]:

valid_losses = []
train_losses = []
valid_accuracies = []
valid_loss_min = np.Inf

model = model.fix_precision().share(ada, bob, crypto_provider=crypto_provdr, requires_grad=True)
optimizer = optimizer.fix_precision()

for epoch in range(1, args.epochs + 1):
    
    # initialize variables to monitor training and validation loss
    training_loss = 0.0
    training_accuracy = 0.0

    if scheduler is not None:
        scheduler.step()
        
    training_loss =  train(args, model, device, private_train_loader, optimizer, epoch)
    validation_loss, validation_accuracy = valid_encrypted(args, model, device, private_valid_loader)

    #if scheduler is not None:
      #scheduler.step(validation_loss) # in case of ReduceOnPlateau 

    ###### print training/validation statistics 
    train_losses.append(training_loss)
    valid_losses.append(validation_loss)
    valid_accuracies.append(validation_accuracy)

    #hour, minute, second = get_time()
    print('Epoch: {} \tTrain. Loss: {:.6f} \tValid. Loss: {:.6f} \t Accur.: {:.6f}'.format(
              epoch,
              training_loss,
              validation_loss,
              validation_accuracy ))

    ###### TODO: save the model if validation loss has decreased
    if validation_loss <= valid_loss_min:
        print('Validation loss decreased by {:.6f}'.format(validation_loss - valid_loss_min))
        #torch.save(model.state_dict(), save_path)
        valid_loss_min = validation_loss
    
##### visualize
plot_loss_acc(args.epochs, train_losses, valid_losses, valid_accuracies)

In [0]:

print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}')

#ada.clear_objects()
#bob.clear_objects()
#cyd.clear_objects()